In [1]:
gROOT->ProcessLine(".L /home/rj/app/lib/libRadioScatter.so");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/RadioScatter.hh>");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/RSEventSummary.hh>");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/RadioScatterEvent.hh>");
gROOT->ProcessLine("#include </home/rj/app/include/RadioScatter/TUtilRadioScatter.hh>");
TString infile = "/home/rj/RadioScatter/doc/smallmultiscat_0MHz_10W_10ns.root";
auto ff=TFile::Open(infile, "READ");
TFile *outfile = new TFile("/home/rj/RadioScatter/outputfiles/voltage_time_angle.root", "RECREATE");
TTree *outtree = new TTree("tree", "test label");

int rxindex = 0;
int txindex = 0;
int antenna = 0;
int entry = 0;
auto tree=(TTree*)ff->Get("tree");

In [2]:
auto event= new RadioScatterEvent();
tree->SetBranchAddress("event", &event);
tree->GetEntry(29);
// tree->GetEntries();
// cout << event->nrx;
auto c1 = TUtilRadioScatter::canvas();
TCanvas *c2 = new TCanvas("c2","c2",200,10,1200,1000);

In [3]:
double* getpows(int t_window, double* powarray, TH2D *spec){
    int NumFreqBins = spec->GetNbinsX();
    int NumIntensBins = spec->GetNbinsY();
    int binmax = spec->GetMaximumBin(); // bin number of max intensity
    double *pows = new double[NumFreqBins]; // array of powers
    double *ybin = new double[NumFreqBins]; // array of y bin numbers
    double_t iterator = 0; // iterator for the for loop
    int i = t_window - 390; // time bin number of the max intensity
    for(int j = 0; j < spec->GetNbinsY(); j++){
        if(j==0){pows[j]=0;}
        else{pows[j]=spec->GetBinContent(i,j);}
        ybin[j] = j*0.015625*1000; // y bin number x YBinWidth * 1000MHz/GHz
        iterator ++;
    }
//    TGraph *gr = new TGraph(NumFreqBins, ybin, pows);
    //return pows and ybin
    return pows;
}

In [6]:
TString infile = "/home/rj/RadioScatter/doc/smallmultiscat_0MHz_10W_10ns.root";
    auto ff = TFile::Open(infile, "READ");

    TCanvas *c1 = new TCanvas("c1", "c1", 800, 600);
    c1->Divide(2, 2);
//    c1->SetWindowSize(1200, 1200);
    c1->SetWindowSize(1200, 800);

//    c1->cd(1)->SetGrid();
    gPad->SetLeftMargin(.1);
    gPad->SetBottomMargin(.1);
    c1->cd(1);
//    TPad *pad1 = new TPad("pad1", "pad1", 0, 0.3, 1, 1);
//    pad1->SetBottomMargin(0);
//    pad1->Draw();
//    pad1->cd();
//    TPad *pad2 = new TPad("pad2", "pad2", 0, 0, 1, 0.3);
//    pad2->SetTopMargin(0);
    c1->cd(1)->SetLeftMargin(.15);
    c1->cd(1)->SetRightMargin(.2);
    c1->cd(2)->SetLeftMargin(.15);
//    pad2->SetRightMargin(.5);
//    pad2->SetBottomMargin(0.2);
//    pad2->Draw();
//    pad2->cd();

    int rxindex = 0;
    int txindex = 0;
    int antenna = 0;
    int entry = 0;
    auto tree = (TTree *) ff->Get("tree");
    auto event = new RadioScatterEvent();
    tree->SetBranchAddress("event", &event);
    tree->GetEntry(entry);
    int entries = tree->GetEntries();
// cout << event->nrx;
//    auto c1 = TUtilRadioScatter::canvas();
//    c1->Clear();
    auto evG = event->getGraph(rxindex, txindex);
    TUtilRadioScatter::titles(evG, "", "Time [ns]", "V");
    TUtilRadioScatter::style(evG, kBlack, 1, 1);
    TUtilRadioScatter::xrange(evG, evG->GetX()[0], evG->GetX()[evG->GetN() - 1]);
    int bins = 32, overlap = 31;
    auto spec = TUtilRadioScatter::FFT::spectrogram(evG, bins, overlap, bins * 2, 2, 0, 0, .2);
    c1->SetLogy(0);
// evG->Draw();
    c1->cd(1);
    c1->cd(1)->SetLogy(0);
    spec->SetStats(0);
    spec->Draw("colz");
//    c1->Draw();

    int  time_window = 479;
    int i = time_window - 390;

    auto mg = new TMultiGraph();
    c1->cd(2)->SetLogy(1);
    int binmax = spec->GetMaximumBin()-390; // bin number of max intensity
    cout << "max bin number: " << binmax << "ns" << endl; // 390 is the time bin number of the first bin

    int NumFreqBins = spec->GetNbinsX();
    int NumIntensBins = spec->GetNbinsY();
    int binmax = spec->GetMaximumBin(); // bin number of max intensity
    double *pows = new double[NumFreqBins]; // array of powers
    double *ybin = new double[NumFreqBins]; // array of y bin numbers
    double_t iterator = 0; // iterator for the for loop
    int i = t_window - 390; // time bin number of the max intensity

    //initialize pows with zero values
    for(int j = 0; j < spec->GetNbinsY(); j++){
        pows[j]=0;
    }
    // loops through entries in the tree
    for(int k = 0; k < entries; k++){
        tree->GetEntry(k);


    for(int j = 0; j < spec->GetNbinsY(); j++){
        if(j==0){pows[j]=0;}
        else{pows[j]=spec->GetBinContent(i,j);}
        ybin[j] = j*0.015625*1000; // y bin number x YBinWidth * 1000MHz/GHz
        iterator ++;


    TGraph *gr = new TGraph(NumFreqBins, ybin, freqs);
    gr->GetYaxis()->SetRangeUser(1*pow(10,-16.5), 1*pow(10,-11));
    gr->SetMarkerColor(4);
    gr->SetMarkerSize(.5);
    gr->SetMarkerStyle(20);
    gr->GetXaxis()->SetLimits(0,200);
    gr->SetTitle(Form("Frequency vs Power at time %dns", 390+i));
    gr->GetXaxis()->SetTitle("");
    gr->GetYaxis()->SetTitle("WGHz^{-1} x 10^{-12}");
    gr->Draw("ACP");
    c1->Draw();

Unbalanced braces. This cell was not processed.
